In [109]:
import pandas as pd 

df=pd.read_csv("25-27_data.csv")
city=list(df["Cities"].str.split(","))

# to capture date and month
dept_date=[]
dept_month=[]
date=[x[1] for x in city]
for i in date:
    k=i.strip()
    dept_date.append(k.split(" ")[0])
    dept_month.append(k.split(" ")[1])
dept_city=[]
arrival_city=[]

# to capture departure city and arrival city
c=[x[0] for x in city]
for i in c:
    dept_city.append(i.split(" to ")[0])
    arrival_city.append(i.split(" to ")[1])

# to capture the number of stops 
stops=[]
for i in df["layeover"]:
    if i[0]=="N":
        stops.append(0)
    else:
        stops.append(i[0])

        
# to capture the layover cities
layover_cities=[]
for i in df["layeover"]:
    if "via" in i:
        layover_cities.append(i.split(" via ")[1])
    else:
        layover_cities.append("No Stops")
        
#creating dataframe
df["Dept_city"]=dept_city
df["Dept_date"]=dept_date
df["Dept_month"]=dept_month
df["arrival_city"]=arrival_city
df["stops"]=stops
df["layover_cities"]=layover_cities


# droping the usless columns
df.drop(columns=["Cities","layeover"],axis=1,inplace=True)

In [110]:
df.head()

,Price,departure_time,arrival_time,Airline,Cabin,Dept_city,Dept_date,Dept_month,arrival_city,stops,layover_cities
0,"2,382",19:30,20:30,Air India,E,Chandigarh,01,May,New Delhi,0,No Stops
1,"2,382",21:15,22:15,Air India,E,Chandigarh,01,May,New Delhi,0,No Stops
2,"2,854",07:05,08:00,Air India,E,Chandigarh,01,May,New Delhi,0,No Stops
3,"2,854",07:35,08:30,IndiGo,E,Chandigarh,01,May,New Delhi,0,No Stops
4,"2,854",14:10,15:10,IndiGo,E,Chandigarh,01,May,New Delhi,0,No Stops


In [111]:
df.shape

(3654, 11)

In [112]:
df.drop_duplicates(inplace=True)

In [113]:
# more columns can be added such as duration,year,weekday,dayofweek ,class of the seat

In [114]:
#Creating Departure Date Columns

dates1=[]
for i in range(df.shape[0]):
    x="2021-05-"+str(df["Dept_date"].iloc[i])+" "+str(df["departure_time"].iloc[i])
    dates1.append(x)
df["Departure_Date"]=pd.to_datetime(dates1)

In [115]:
#Creating Arrival Date Columns
dates1=[]
for i in range(df.shape[0]):
    x="2021-05-"+str(df["Dept_date"].iloc[i])+" "+str(df["arrival_time"].iloc[i])
    dates1.append(x)
df["Arrival_Date"]=pd.to_datetime(dates1)

In [116]:
#Creating Duration Column
import numpy as np

df["duration_time"]=df["Arrival_Date"]-df["Departure_Date"]
df["duration"]=df["duration_time"]/ np.timedelta64(1, 'm')
for i in range(df.shape[0]):
    if df["duration"].iloc[i]<0:
        df["duration"].iloc[i]=1440+df["duration"].iloc[i]

C:\Users\Dell\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [117]:
#Creating columns like Weekday/Weekname
df["weekday"]=pd.to_datetime(df["Departure_Date"]).dt.weekday
df["day_name"]=pd.to_datetime(df["Departure_Date"]).dt.day_name()

In [118]:
#Creating columns like wehther its a morning,evening,night or afternoon flight from departure time & arrival time of the flight
df["dept_hours"]=pd.to_datetime(df["Departure_Date"]).dt.hour
df["arrival_hours"]=pd.to_datetime(df["Arrival_Date"]).dt.hour

def depart(x):
    d=[]
    for i in range(df.shape[0]):
        if x.iloc[i]>=16 and x.iloc[i]<21:
            d.append("Evening_flight")
    
        if x.iloc[i]>=21 or x.iloc[i]<5:
            d.append("Night_flight")
    
        if x.iloc[i]>=5 and x.iloc[i]<11:
            d.append("Morning_flight")
    
        if x.iloc[i]>=11 and x.iloc[i]<16:
            d.append("Afternoon_flight")
    return d

df["Dept_flights_time"]=depart(df["dept_hours"])
df["arrival_flights_time"]=depart(df["arrival_hours"])


In [119]:
df.drop(columns=["departure_time","arrival_time","duration_time"],axis=1,inplace=True)

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3265 entries, 0 to 3653
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Price                 3265 non-null   object        
 1   Airline               3265 non-null   object        
 2   Cabin                 3265 non-null   object        
 3   Dept_city             3265 non-null   object        
 4   Dept_date             3265 non-null   object        
 5   Dept_month            3265 non-null   object        
 6   arrival_city          3265 non-null   object        
 7   stops                 3265 non-null   object        
 8   layover_cities        3265 non-null   object        
 9   Departure_Date        3265 non-null   datetime64[ns]
 10  Arrival_Date          3265 non-null   datetime64[ns]
 11  duration              3265 non-null   float64       
 12  weekday               3265 non-null   int64         
 13  day_name          

In [121]:
df["weekday"].value_counts()

6    1048
5     750
3     428
2     416
0     244
1     225
4     154
Name: weekday, dtype: int64

In [122]:
df["Price"]=df["Price"].str.replace(",","")
df["Price"]=df["Price"].astype("int")

In [123]:
df.head()

,Price,Airline,Cabin,Dept_city,Dept_date,Dept_month,arrival_city,stops,layover_cities,Departure_Date,Arrival_Date,duration,weekday,day_name,dept_hours,arrival_hours,Dept_flights_time,arrival_flights_time
0,2382,Air India,E,Chandigarh,01,May,New Delhi,0,No Stops,2021-05-01 19:30:00,2021-05-01 20:30:00,60.0,5,Saturday,19,20,Evening_flight,Evening_flight
1,2382,Air India,E,Chandigarh,01,May,New Delhi,0,No Stops,2021-05-01 21:15:00,2021-05-01 22:15:00,60.0,5,Saturday,21,22,Night_flight,Night_flight
2,2854,Air India,E,Chandigarh,01,May,New Delhi,0,No Stops,2021-05-01 07:05:00,2021-05-01 08:00:00,55.0,5,Saturday,7,8,Morning_flight,Morning_flight
3,2854,IndiGo,E,Chandigarh,01,May,New Delhi,0,No Stops,2021-05-01 07:35:00,2021-05-01 08:30:00,55.0,5,Saturday,7,8,Morning_flight,Morning_flight
4,2854,IndiGo,E,Chandigarh,01,May,New Delhi,0,No Stops,2021-05-01 14:10:00,2021-05-01 15:10:00,60.0,5,Saturday,14,15,Afternoon_flight,Afternoon_flight


In [124]:
df["stops"]=df["stops"].astype("int8")
df["weekday"]=df["weekday"].astype("int8")
df["duration"]=df["duration"].astype("float16")
df["dept_hours"]=df["dept_hours"].astype("int8")
df["arrival_hours"]=df["arrival_hours"].astype("int8")
df["Dept_date"]=df["Dept_date"].astype("int8")

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3265 entries, 0 to 3653
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Price                 3265 non-null   int32         
 1   Airline               3265 non-null   object        
 2   Cabin                 3265 non-null   object        
 3   Dept_city             3265 non-null   object        
 4   Dept_date             3265 non-null   int8          
 5   Dept_month            3265 non-null   object        
 6   arrival_city          3265 non-null   object        
 7   stops                 3265 non-null   int8          
 8   layover_cities        3265 non-null   object        
 9   Departure_Date        3265 non-null   datetime64[ns]
 10  Arrival_Date          3265 non-null   datetime64[ns]
 11  duration              3265 non-null   float16       
 12  weekday               3265 non-null   int8          
 13  day_name          

In [126]:
df.to_csv("15-20_ready.csv",index=False)

In [127]:
df1 = pd.read_csv("15-20_ready.csv")
df1.head()

,Price,Airline,Cabin,Dept_city,Dept_date,Dept_month,arrival_city,stops,layover_cities,Departure_Date,Arrival_Date,duration,weekday,day_name,dept_hours,arrival_hours,Dept_flights_time,arrival_flights_time
0,2382,Air India,E,Chandigarh,1,May,New Delhi,0,No Stops,2021-05-01 19:30:00,2021-05-01 20:30:00,60.0,5,Saturday,19,20,Evening_flight,Evening_flight
1,2382,Air India,E,Chandigarh,1,May,New Delhi,0,No Stops,2021-05-01 21:15:00,2021-05-01 22:15:00,60.0,5,Saturday,21,22,Night_flight,Night_flight
2,2854,Air India,E,Chandigarh,1,May,New Delhi,0,No Stops,2021-05-01 07:05:00,2021-05-01 08:00:00,55.0,5,Saturday,7,8,Morning_flight,Morning_flight
3,2854,IndiGo,E,Chandigarh,1,May,New Delhi,0,No Stops,2021-05-01 07:35:00,2021-05-01 08:30:00,55.0,5,Saturday,7,8,Morning_flight,Morning_flight
4,2854,IndiGo,E,Chandigarh,1,May,New Delhi,0,No Stops,2021-05-01 14:10:00,2021-05-01 15:10:00,60.0,5,Saturday,14,15,Afternoon_flight,Afternoon_flight


In [128]:
df1.shape

(3265, 18)

In [129]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3265 entries, 0 to 3264
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Price                 3265 non-null   int64  
 1   Airline               3265 non-null   object 
 2   Cabin                 3265 non-null   object 
 3   Dept_city             3265 non-null   object 
 4   Dept_date             3265 non-null   int64  
 5   Dept_month            3265 non-null   object 
 6   arrival_city          3265 non-null   object 
 7   stops                 3265 non-null   int64  
 8   layover_cities        3265 non-null   object 
 9   Departure_Date        3265 non-null   object 
 10  Arrival_Date          3265 non-null   object 
 11  duration              3265 non-null   float64
 12  weekday               3265 non-null   int64  
 13  day_name              3265 non-null   object 
 14  dept_hours            3265 non-null   int64  
 15  arrival_hours        

In [130]:
df2 = pd.read_csv("ready_dataset.csv", sep=',')
df2.head()

,Price,Airline,Cabin,Dept_city,Dept_date,Dept_month,arrival_city,stops,layover_cities,Departure_Date,Arrival_Date,duration,weekday,day_name,dept_hours,arrival_hours,Dept_flights_time,arrival_flights_time
0,6159,Vistara,PE,New Delhi,20,Apr,Mumbai,0,No Stops,20-04-2021 09:30,20-04-2021 11:40,130,1,Tuesday,9,11,Morning_flight,Afternoon_flight
1,6159,Vistara,PE,New Delhi,20,Apr,Mumbai,0,No Stops,20-04-2021 15:30,20-04-2021 17:40,130,1,Tuesday,15,17,Afternoon_flight,Evening_flight
2,6159,Vistara,PE,New Delhi,20,Apr,Mumbai,0,No Stops,20-04-2021 16:55,20-04-2021 19:05,130,1,Tuesday,16,19,Evening_flight,Evening_flight
3,6159,Vistara,PE,New Delhi,20,Apr,Mumbai,0,No Stops,20-04-2021 12:45,20-04-2021 15:00,135,1,Tuesday,12,15,Afternoon_flight,Afternoon_flight
4,6159,Vistara,PE,New Delhi,20,Apr,Mumbai,0,No Stops,20-04-2021 17:45,20-04-2021 20:05,140,1,Tuesday,17,20,Evening_flight,Evening_flight


In [131]:
df2 = df2.append(df1)

In [132]:
df2.to_csv("final_dataset.csv",index=False)

In [133]:
df3 = pd.read_csv("final_dataset.csv")

In [134]:
df3.shape

(325484, 18)

In [135]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325484 entries, 0 to 325483
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Price                 325484 non-null  int64  
 1   Airline               325484 non-null  object 
 2   Cabin                 325484 non-null  object 
 3   Dept_city             325484 non-null  object 
 4   Dept_date             325484 non-null  int64  
 5   Dept_month            325484 non-null  object 
 6   arrival_city          325484 non-null  object 
 7   stops                 325484 non-null  int64  
 8   layover_cities        325484 non-null  object 
 9   Departure_Date        325484 non-null  object 
 10  Arrival_Date          325484 non-null  object 
 11  duration              325484 non-null  float64
 12  weekday               325484 non-null  int64  
 13  day_name              325484 non-null  object 
 14  dept_hours            325484 non-null  int64  
 15  

In [136]:
df3.columns

Index(['Price', 'Airline', 'Cabin', 'Dept_city', 'Dept_date', 'Dept_month',
       'arrival_city', 'stops', 'layover_cities', 'Departure_Date',
       'Arrival_Date', 'duration', 'weekday', 'day_name', 'dept_hours',
       'arrival_hours', 'Dept_flights_time', 'arrival_flights_time'],
      dtype='object')

In [137]:
df2.columns

Index(['Price', 'Airline', 'Cabin', 'Dept_city', 'Dept_date', 'Dept_month',
       'arrival_city', 'stops', 'layover_cities', 'Departure_Date',
       'Arrival_Date', 'duration', 'weekday', 'day_name', 'dept_hours',
       'arrival_hours', 'Dept_flights_time', 'arrival_flights_time'],
      dtype='object')

In [138]:
df3.tail()

,Price,Airline,Cabin,Dept_city,Dept_date,Dept_month,arrival_city,stops,layover_cities,Departure_Date,Arrival_Date,duration,weekday,day_name,dept_hours,arrival_hours,Dept_flights_time,arrival_flights_time
325479,9114,Vistara,PE,Raipur,16,May,Srinagar,1,New Delhi,2021-05-16 08:55:00,2021-05-16 13:10:00,255.0,6,Sunday,8,13,Morning_flight,Afternoon_flight
325480,9674,Vistara,PE,Raipur,16,May,Ranchi,1,New Delhi,2021-05-16 08:55:00,2021-05-16 19:40:00,645.0,6,Sunday,8,19,Morning_flight,Evening_flight
325481,9674,Vistara,PE,Raipur,16,May,Ranchi,1,New Delhi,2021-05-16 20:20:00,2021-05-16 08:20:00,720.0,6,Sunday,20,8,Evening_flight,Morning_flight
325482,14602,Vistara,PE,Raipur,16,May,Port Blair,2,"New Delhi,Kolkata",2021-05-16 20:20:00,2021-05-16 11:20:00,900.0,6,Sunday,20,11,Evening_flight,Afternoon_flight
325483,15554,Vistara,PE,Raipur,16,May,Port Blair,2,"New Delhi,Kolkata",2021-05-16 08:55:00,2021-05-16 11:20:00,145.0,6,Sunday,8,11,Morning_flight,Afternoon_flight


In [139]:
df3.shape

(325484, 18)

In [140]:
df3.drop_duplicates(inplace=True)

In [141]:
df3.to_csv("final_dataset.csv",index=False)

In [142]:
df4 = pd.read_csv("final_dataset.csv")

In [143]:
df4.shape

(315985, 18)

In [145]:
df4.isnull().sum()

Price                   0
Airline                 0
Cabin                   0
Dept_city               0
Dept_date               0
Dept_month              0
arrival_city            0
stops                   0
layover_cities          0
Departure_Date          0
Arrival_Date            0
duration                0
weekday                 0
day_name                0
dept_hours              0
arrival_hours           0
Dept_flights_time       0
arrival_flights_time    0
dtype: int64